# 2. Conjuctive and disjunctive conditions

In previous versions of eFLINT, `Conditioned by` clauses could only be associated with actions to determine whether these actions are `Enabled`, i.e. do *not* cause violations when they are triggered. Experiments showed the value of impliciting using logical conjunction to combine multiple conditions expressed using `Conditioned by`, whether as part of a single declaration or through the use of `Extend`. The combination of the *conjuctive*  `Conditioned by` clauses and the *disjunctive* `Holds when` (and `Derived from`) clauses proved very expressive, in a way that is also useful for other kinds of types. 

In eFLINT-3.0, the role of `Conditioned by` clauses has changed. These clauses can now also be associated with the other kinds of types, i.e. fact-types, duty-types and event-types. Moreover, they are not only used to determine whether an instance of a type is `Enabled`, but act as a kind of filter on derivation rules. That is, an instance of a type is considered to be *derivable* if one or more of its derivation rules say it is **and** if all conditions of the type are satisfied. (As a consequence, the only difference between `Holds` and `Enabled` is on instances whose truth is postulated rather than derived and whose type has one or more conditions associated with it.)

To demonstrate the expressiveness of conjuctive and disjunctive conditions this notebook works out an example of an exception on an exception. 

Consider the following article about entry requirements on a school:

> Article 1) An Applicant can be accepted into St. John's Academy only if they have completed a Primary School with a GPA score of at least 3 as demonstrated by a nationally recognised Diploma.

This article lays out a condition (GPA >= 3) that applies to any potential student, irrespective of the source of their eligibilty. Therefore the condition is to be expressed as a conjunctive clause.

In [1]:
Fact applicant
Fact primary-school Identified by StMary, StGeorge
Fact gpa Identified by 1..4
Fact diploma Identified by primary-school * applicant * gpa 
Fact accepted Identified by applicant

Act accept-application Recipient applicant
  Holds when applicant                 // contextual condition for the power
  Conditioned by diploma() && gpa >= 3 // a valid diploma must exist for this applicant with gpa >= 3
  Creates accepted()
.
+applicant(Alice).
+applicant(Bob).
+applicant(Chloe).
+diploma(StMary, Alice, 3).
+diploma(StGeorge, Bob, 3).
+diploma(StGeorge, Chloe, 2).
?Enabled(accept-application(StJohn, Alice)).
?Enabled(accept-application(StJohn, Bob)).
?!Enabled(accept-application(StJohn, Chloe)).

+applicant("Alice")
+applicant("Bob")
+applicant("Chloe")
+diploma(primary-school("StGeorge"),applicant("Bob"),gpa(3))
+diploma(primary-school("StGeorge"),applicant("Chloe"),gpa(2))
+diploma(primary-school("StMary"),applicant("Alice"),gpa(3))


Query success
Query success
Query success


The code cell above shows the eligibility condition of article 1 as a `Conditioned by` on the act-type `accept-application`. Written as such, conditioned can not be 'weakened' without modifying the original code, i.e. by executing an additional declaration. A layer of indirection can be used to avoid this problem by adding a fact-type (`[Article 1]` below) that has its own conditions and derivation rules.

In [2]:
Act accept-application Recipient applicant
  Holds when applicant // contextual condition for the power
  Conditioned by [Article 1]()
  
Fact [Article 1] 
  Identified by applicant // identified by all the types bound on the 'call site'
  Holds when diploma() && gpa >= 3
.
?Enabled(accept-application(StJohn, Alice)).
?Enabled(accept-application(StJohn, Bob)).
?!Enabled(accept-application(StJohn, Chloe)).

+[Article 1](applicant("Alice"))
+[Article 1](applicant("Bob"))


Query success
Query success
Query success


Because the condition expressed in article 1 is now written in a `Holds when` clause, it can be overruled by additional `Holds when` clauses added to the type `[Article 1]`. However, because the article 1 condition is still referred to from a `Conditioned by` clause in `accept-application`, it is still a condition that needs to hold true for all applications that are to be accepted. To see how this works, consider the following exception to article 1 and its formalisation.

> Article 2) An exception to Article 1 can be made for all applicants with a Diploma from St. George's Primary School with a GPA of at least 2

In [3]:
Extend Fact [Article 1] // condition that turns article 1 "off"
  Holds when [Article 2]()
  
Fact [Article 2] Identified by applicant
  Holds when diploma() && primary-school == StGeorge && gpa >= 2
.
accept-application(StJohn, Alice).
accept-application(StJohn, Bob).
?Enabled(accept-application(StJohn, Chloe)).

+[Article 2](applicant("Bob"))
+[Article 1](applicant("Chloe"))
+[Article 2](applicant("Chloe"))


Executed transition: accept-application(actor("StJohn"),applicant("Alice"))
Executed transition: accept-application(actor("StJohn"),applicant("Bob"))
Query success


Exeptions on Article 2 are possible because the Article 2 exception is itself applied through the indirection suggested for Article 1. For example,

> Article 3) However, Article 2 is only applicable for the first applicant from St George's.

In [4]:
Extend Fact [Article 2] 
  Conditioned by Not([Article 3]())
  
Fact [Article 3] Identified by applicant
  Holds when Not(Exists applicant : accepted() && diploma(primary-school=StGeorge))
.
?!Enabled(accept-application(StJohn, Chloe)).

~[Article 2](applicant("Bob"))
~[Article 1](applicant("Chloe"))
~[Article 2](applicant("Chloe"))
+[Article 3](applicant("Alice"))
+[Article 3](applicant("Bob"))
+[Article 3](applicant("Chloe"))


Query success


Note that Article 2 is a positive exception and therefore results in an additional `Holds when` clause (disjunctive condition) and that Article 3 is a negative exception and therefore results in an additional `Conditioned by` clause (conjunctive conditions).